In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import nltk
import spacy
from nltk.corpus import stopwords
import string

In [ ]:
df = pd.read_csv('../data/process_v1/process_data_4.csv')

In [ ]:
df

In [ ]:
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
	s = ''
	for c in input_str:
		if c in s1:
			s += s0[s1.index(c)]
		else:
			s += c
	return s

streets = [remove_accents(item) for item in df['street'].unique()]

In [ ]:
address_words = []

for street in streets:
    address_words += street.split(" ")
address_words[0]

In [ ]:
eng_1_df = pd.read_csv('../data/process_v1/trans/eng_1_20000.csv')
eng_2_df = pd.read_csv('../data/process_v1/trans/eng_2_20000.csv')
eng_3_df = pd.read_csv('../data/process_v1/trans/eng_3_20000.csv')
eng_4_df = pd.read_csv('../data/process_v1/trans/eng_4_20000.csv')
eng_5_df = pd.read_csv('../data/process_v1/trans/eng_5_20000.csv')
eng_6_df = pd.read_csv('../data/process_v1/trans/eng_6_20000.csv')
eng_7_df = pd.read_csv('../data/process_v1/trans/eng_7_20000.csv')
eng_8_df = pd.read_csv('../data/process_v1/trans/eng_8_30000.csv')
eng_9_df = pd.read_csv('../data/process_v1/trans/eng_9.csv')


In [ ]:
eng_df = pd.concat([eng_1_df, eng_2_df, eng_3_df, eng_4_df, eng_5_df,eng_6_df ,eng_7_df, eng_8_df, eng_9_df])
eng_df

In [ ]:
eng_df['en_description'] = eng_df['en_description'].fillna("No information")

In [ ]:
eng_df["text_lower"] = eng_df["en_description"].str.lower()
eng_df.head()

In [ ]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

eng_df["text_wo_punct"] = eng_df["text_lower"].apply(lambda text: remove_punctuation(text))
eng_df.head()

In [ ]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

eng_df["text_wo_stop"] = eng_df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
eng_df.head()

In [ ]:
from collections import Counter
cnt = Counter()
for text in eng_df["text_wo_stop"].values:
    for word in text.split():
        cnt[word] += 1

In [ ]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

eng_df["text_wo_stopfreq"] = eng_df["text_wo_stop"].apply(lambda text: remove_freqwords(text))
eng_df.head()

In [ ]:
len(cnt.most_common())

In [ ]:
n_rare_words = 40000
RAREWORDS = [w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]]
NUM_WORDS = [w for (w, wc) in cnt.most_common() if sum([_.isnumeric() for _ in w])]
ADDRESS_WORDS = [w for (w, wc) in cnt.most_common() if w in address_words]

RAREWORDS = set(RAREWORDS + NUM_WORDS + ADDRESS_WORDS)
print(RAREWORDS)

In [ ]:
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

eng_df["text_wo_stopfreqrare"] = eng_df["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
eng_df.head()

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

eng_df["text_wo_stopfreqrare"] = eng_df["text_wo_stopfreqrare"].apply(remove_urls)

In [ ]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)


eng_df["text_wo_stopfreqrare"] = eng_df["text_wo_stopfreqrare"].apply(remove_html)

In [ ]:
eng_df['text_wo_stopfreqrare']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn.model_selection import train_test_split


data_train, data_test = train_test_split(eng_df['text_wo_stopfreqrare'], test_size=0.2, random_state=42)

In [ ]:
training_corpus = data_train.values

vectorizer = CountVectorizer(min_df=3)

vectorizer.fit(training_corpus)
num_words = len(vectorizer.vocabulary_)
print("Number of distinct words to be used as features:",num_words)

In [24]:
vectorizer.vocabulary_

{'apartment': 482,
 'building': 1739,
 'location': 7816,
 'densely': 3378,
 'populated': 10237,
 'dong': 4058,
 'district': 3667,
 'hanoi': 5965,
 'clear': 2404,
 'alley': 298,
 'facing': 4856,
 'main': 8040,
 'extremely': 4830,
 'convenient': 2861,
 'transportation': 13794,
 'near': 8880,
 'series': 11948,
 'roads': 11439,
 'surrounded': 13096,
 'residential': 11241,
 'markets': 8140,
 'general': 5552,
 'hospitals': 6353,
 'national': 8862,
 'standard': 12668,
 'schools': 11732,
 'levels': 7629,
 'successful': 12962,
 'mens': 8308,
 'office': 9320,
 'buildings': 1743,
 'large': 7475,
 'offices': 9332,
 'chamvit': 2170,
 'lotte': 7912,
 'vincom': 14456,
 'universities': 14111,
 'many': 8107,
 'students': 12884,
 'culture': 3119,
 'labor': 7374,
 'war': 14653,
 'invalids': 7018,
 'social': 12346,
 'affairs': 196,
 'foreign': 5232,
 'trade': 13678,
 'diplomacy': 3568,
 'law': 7514,
 'university': 14116,
 'nearly': 8886,
 'square': 12604,
 'meters': 8342,
 'design': 3428,
 'solid': 12377,